# Example: convert WRF LES output to DEPHY format
# Code to read WRF LES output files and write to DEPHY format (NetCDF)
### Contributed by Tim Juliano from NCAR on 5/17/2023
### Based on code from Ann Fridlind from NASA/GISS

### Import libraries

In [8]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import netCDF4
import datetime as dt
from netCDF4 import Dataset
import glob
#from wrf import destagger

### Read WRF domain-mean output files

In [9]:
# specify source directory
my_rootdir = '/glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/'
my_simname = 'Lx25km_dx100m'
my_rundir = my_rootdir + my_simname
my_subdirs = sorted(glob.glob(my_rundir + '/run*/'))


# dephy save name
dephy_filename = 'WRF_LES_COMBLE-I_' + my_simname + '.nc'

# read in WRF parameter settings
wrfout_filenames = sorted(glob.glob(my_subdirs[0] + 'wrfout*'))
wrfout_params = xr.open_dataset(wrfout_filenames[0],decode_times=False)
wrf_dx = wrfout_params.DX
wrf_dy = wrfout_params.DY
wrf_nx = wrfout_params.dims['west_east']
wrf_ny = wrfout_params.dims['south_north']
wrf_nz = wrfout_params.dims['bottom_top']
wrf_lat = wrfout_params['XLAT'].data.mean()

# specify save directory
my_savedir = './output_les/wrf/round4/' + my_simname + '/'
if not os.path.exists(my_savedir):
    os.makedirs(my_savedir)

In [10]:
# read in WRF files, which contain all of our information
for i in np.arange(len(my_subdirs)):
    if i == 0:
        input_filenames = sorted(glob.glob(my_subdirs[i] + 'wrfstat*'))
    else:
        input_filenames = np.concatenate((input_filenames,sorted(glob.glob(my_subdirs[i] + 'wrfstat*'))))
print (input_filenames)

['/glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run1/wrfstat_d01_2020-03-12_22:00:00'
 '/glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run1/wrfstat_d01_2020-03-12_23:00:00'
 '/glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run1/wrfstat_d01_2020-03-13_00:00:00'
 '/glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run2/wrfstat_d01_2020-03-13_00:00:00'
 '/glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run2/wrfstat_d01_2020-03-13_01:00:00'
 '/glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run2/wrfstat_d01_2020-03-13_02:00:00'
 '/glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF

In [11]:
wrf_file = xr.open_dataset(input_filenames[0],decode_times=False)
    
# find all WRF variables
wrf_vars = [i for i in wrf_file.data_vars]
#print (wrf_vars)

# find subset of variables for 1D time series (these variables start with 'CST')
wrf_vars_cst = [j for j in wrf_vars if ('CST' in j or 'Times' in j)]

# find subset of variables for 2D (time x height) soundings (these variables start with 'CSP')
wrf_vars_csp = [j for j in wrf_vars if ('CSP' in j or 'Times' in j)]

In [12]:
# read in WRF domain-mean profiles
skip_t0_next_file = False
for i in np.arange(len(input_filenames)):
    print ('Doing ', input_filenames[i])
    dummy_file = xr.open_dataset(input_filenames[i],decode_times=False)
    if i == 0:
        wrf_snds = dummy_file[wrf_vars_csp]
    else:
        if skip_t0_next_file:
            wrf_snds = xr.concat([wrf_snds,dummy_file[wrf_vars_csp].isel(Time=np.arange(1,12))],dim="Time")
            skip_t0_next_file = False
        else:
            wrf_snds = xr.concat([wrf_snds,dummy_file[wrf_vars_csp]],dim="Time")
            
    if len(dummy_file['Times']) == 1:
        skip_t0_next_file = True

Doing  /glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run1/wrfstat_d01_2020-03-12_22:00:00
Doing  /glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run1/wrfstat_d01_2020-03-12_23:00:00
Doing  /glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run1/wrfstat_d01_2020-03-13_00:00:00
Doing  /glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run2/wrfstat_d01_2020-03-13_00:00:00
Doing  /glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run2/wrfstat_d01_2020-03-13_01:00:00
Doing  /glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run2/wrfstat_d01_2020-03-13_02:00:00
Doing  /glade/scratch/tjuliano/doe_combl

In [13]:
# read in WRF domain-mean scalars
skip_t0_next_file = False
for i in np.arange(len(input_filenames)):
    print ('Doing ', input_filenames[i])
    dummy_file = xr.open_dataset(input_filenames[i],decode_times=False)
    if i == 0:
        wrf_scas = dummy_file[wrf_vars_cst]
    else:
        if skip_t0_next_file:
            wrf_scas = xr.concat([wrf_scas,dummy_file[wrf_vars_cst].isel(Time=np.arange(1,12))],dim="Time")
            skip_t0_next_file = False
        else:
            wrf_scas = xr.concat([wrf_scas,dummy_file[wrf_vars_cst]],dim="Time")        
        
    if len(dummy_file['Times']) == 1:
        skip_t0_next_file = True

Doing  /glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run1/wrfstat_d01_2020-03-12_22:00:00
Doing  /glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run1/wrfstat_d01_2020-03-12_23:00:00
Doing  /glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run1/wrfstat_d01_2020-03-13_00:00:00
Doing  /glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run2/wrfstat_d01_2020-03-13_00:00:00
Doing  /glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run2/wrfstat_d01_2020-03-13_01:00:00
Doing  /glade/scratch/tjuliano/doe_comble/wrf_asr_cao/WRF-ASR-CAO/WRF/test/GOOD_SIMS_1DOM/2023_pan_gass/v2.2/Lx25km_dx100m/run2/wrfstat_d01_2020-03-13_02:00:00
Doing  /glade/scratch/tjuliano/doe_combl

### Unstagger any staggered variables and add to dataset

In [14]:
# vertical velocity
csp_w_mass = (wrf_snds['CSP_W'][:,0:-1].data+wrf_snds['CSP_W'][:,1:].data)/2.
wrf_snds = wrf_snds.assign(variables={'CSP_W_MASS': (('Time', 'bottom_top'), csp_w_mass)})

# vertical velocity variance
csp_w2_mass = (wrf_snds['CSP_W2'][:,0:-1].data+wrf_snds['CSP_W2'][:,1:].data)/2.
wrf_snds = wrf_snds.assign(variables={'CSP_W2_MASS': (('Time', 'bottom_top'), csp_w2_mass)})

### Calculate some additional variables requested, and add them to the xarray

In [15]:
# liquid water path
cst_lwp = wrf_scas['CST_CLWP'].data+wrf_scas['CST_RWP'].data
wrf_scas = wrf_scas.assign(variables={'CST_LWP': (('Time'), cst_lwp)})

# inversion height
cst_zi = np.zeros(np.shape(wrf_snds['CSP_TH'].data)[0])
for i in np.arange(np.shape(wrf_snds['CSP_TH'].data)[0]):
    th_grad = (wrf_snds['CSP_TH'][i,1:120].data - wrf_snds['CSP_TH'][i,0:119].data) / (wrf_snds['CSP_DZ8W'][i,0:119].data)
    th_grad_max = np.argmax(th_grad)
    tke_max = np.max(wrf_snds['CSP_TKE_RS'][i,:].data)
    tke_norm = wrf_snds['CSP_TKE_RS'][i,:].data/tke_max
    tke_idx = np.where(tke_norm<0.2)[0]
    if (wrf_snds['CSP_Z'][i,th_grad_max].data > 0.9*cst_zi[i-1] and wrf_snds['CSP_Z'][i,th_grad_max].data < 1.1*cst_zi[i-1]) or (i<10) :
        cst_zi[i] = wrf_snds['CSP_Z'][i,th_grad_max].data
    else:
        cst_zi[i] = wrf_snds['CSP_Z'][i,tke_idx[0]].data

    #if cst_zi[i] > 4000.:
    #    print (th_grad,th_grad[th_grad_max])
    #    sys.exit()
    
wrf_scas = wrf_scas.assign(variables={'CST_ZI': (('Time'), cst_zi)})

/glade/work/tjuliano/casper_npl_clone/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
/glade/work/tjuliano/casper_npl_clone/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
/glade/work/tjuliano/casper_npl_clone/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in less
  if sys.path[0] == '':


### Read list of requested domain-mean output variables

In [16]:
# read list of requested variables
vars_mean_list = pd.read_excel('https://docs.google.com/spreadsheets/d/1Vl8jYGviet7EtXZuQiitrx4NSkV1x27aJAhxxjBb9zI/export?gid=0&format=xlsx',
                              sheet_name='Mean')
vars_mean_list = vars_mean_list.drop(columns='comment')

# add columns to contain model output name and units conversion factors
vars_mean_list = vars_mean_list.assign(model_name='missing data',conv_factor=1.0)
vars_mean_list

,standard_name,variable_id,units,dimensions,model_name,conv_factor
0,time,time,s,self,missing data,1.0
1,height,zf,m,self,missing data,1.0
2,air_pressure,pa,Pa,"time, height",missing data,1.0
3,air_volumic_mass,rho,kg m-3,"time, height",missing data,1.0
4,air_temperature,ta,K,"time, height",missing data,1.0
...,...,...,...,...,...,...
56,toa_outgoing_longwave_flux,rlut,W m-2,time,missing data,1.0
57,surface_downwelling_longwave_flux,rlds,W m-2,time,missing data,1.0
58,surface_upwelling_longwave_flux,rlus,W m-2,time,missing data,1.0
59,surface_downwelling_shortwave_flux,rsds,W m-2,time,missing data,1.0


### Match WRF scalar variables to requested outputs

In [17]:
# identify requested variables with only time dimension
vars_mean_scas = vars_mean_list[vars_mean_list['dimensions']=='time']

# match to DHARMA variable names and specify conversion factors
for index in vars_mean_scas.index:
    standard_name = vars_mean_list.standard_name.iat[index]
    if standard_name=='surface_upward_sensible_heat_flux': 
        vars_mean_list.model_name.iat[index] = 'CST_SH'
    if standard_name=='surface_upward_latent_heat_flux': 
        vars_mean_list.model_name.iat[index] = 'CST_LH'
    if standard_name=='surface_friction_velocity': 
        vars_mean_list.model_name.iat[index] = 'CST_UST'
    if standard_name=='surface_pressure': 
        vars_mean_list.model_name.iat[index] = 'CST_PS'
    if standard_name=='surface_temperature': 
        vars_mean_list.model_name.iat[index] = 'CST_TSK'
    if standard_name=='inversion_height': 
        vars_mean_list.model_name.iat[index] = 'CST_ZI'
    if standard_name=='cloud_area_fraction': 
        vars_mean_list.model_name.iat[index] = 'CST_CLDTOT2'
    if standard_name=='precipitation_flux_at_surface': 
        vars_mean_list.model_name.iat[index] = 'CST_PRECT'
    #    vars_mean_list.conv_factor.iat[index] = 1/3600.
    if standard_name=='atmosphere_mass_content_of_cloud_water': 
        vars_mean_list.model_name.iat[index] = 'CST_CLWP'
    #    vars_mean_list.conv_factor.iat[index] = 1/1000.
    if standard_name=='atmosphere_mass_content_of_liquid_water': 
        vars_mean_list.model_name.iat[index] = 'CST_LWP'
    #    vars_mean_list.conv_factor.iat[index] = 1/1000.
    if standard_name=='atmosphere_mass_content_of_rain_water': 
        vars_mean_list.model_name.iat[index] = 'CST_RWP'
    if standard_name=='atmosphere_mass_content_of_cloud_ice_water': 
        vars_mean_list.model_name.iat[index] = 'CST_IWP'
    if standard_name=='atmosphere_mass_content_of_snow_water': 
        vars_mean_list.model_name.iat[index] = 'CST_SWP'
    if standard_name=='atmosphere_mass_content_of_graupel_water': 
        vars_mean_list.model_name.iat[index] = 'CST_GWP'
    #    vars_mean_list.conv_factor.iat[index] = 1/1000.
    #if standard_name=='toa_incoming_shortwave_flux': 
    #    vars_mean_list.model_name.iat[index] = 'SWdnTOA'
    #if standard_name=='toa_outgoing_shortwave_flux': 
    #    vars_mean_list.model_name.iat[index] = 'SWupTOA'
    if standard_name=='toa_outgoing_longwave_flux': 
        vars_mean_list.model_name.iat[index] = 'CST_FLNT'
    if standard_name=='surface_upwelling_longwave_flux':
        vars_mean_list.model_name.iat[index] = 'CST_FLNS'
    print(standard_name,' <--- ',vars_mean_list.model_name.iat[index],' * ',vars_mean_list.conv_factor.iat[index])

surface_upward_sensible_heat_flux  <---  CST_SH  *  1.0
surface_upward_latent_heat_flux  <---  CST_LH  *  1.0
obukhov_length  <---  missing data  *  1.0
surface_friction_velocity  <---  CST_UST  *  1.0
inversion_height  <---  CST_ZI  *  1.0
surface_pressure  <---  CST_PS  *  1.0
surface_temperature  <---  CST_TSK  *  1.0
cloud_area_fraction  <---  CST_CLDTOT2  *  1.0
precipitation_flux_at_surface  <---  CST_PRECT  *  1.0
precipitation_flux_at_surface_in_ice_phase  <---  missing data  *  1.0
atmosphere_mass_content_of_cloud_water  <---  CST_CLWP  *  1.0
atmosphere_mass_content_of_rain_water  <---  CST_RWP  *  1.0
atmosphere_mass_content_of_cloud_ice_water  <---  CST_IWP  *  1.0
atmosphere_mass_content_of_snow_water  <---  CST_SWP  *  1.0
atmosphere_mass_content_of_graupel_water  <---  CST_GWP  *  1.0
optical_depth_of_cloud_droplets  <---  missing data  *  1.0
optical_depth  <---  missing data  *  1.0
toa_incoming_shortwave_flux  <---  missing data  *  1.0
toa_outgoing_shortwave_flux  <-

### Match WRF profile variables to requested outputs

In [18]:
# identify requested variables with time and vertical dimensions
vars_mean_snds = vars_mean_list[vars_mean_list['dimensions']=='time, height']

for index in vars_mean_snds.index:
    standard_name = vars_mean_list.standard_name.iat[index]
    if standard_name=='air_pressure': 
        vars_mean_list.model_name.iat[index] = 'CSP_P'
    if standard_name=='air_volumic_mass': 
        vars_mean_list.model_name.iat[index] = 'CSP_RHO'
    #if standard_name=='air_temperature': 
    #    vars_mean_list.model_name.iat[index] = 'T'
    if standard_name=='specific_humidity': 
        vars_mean_list.model_name.iat[index] = 'CSP_QV'
    if standard_name=='total_water_content': 
        vars_mean_list.model_name.iat[index] = 'CSP_QT'
    #if standard_name=='relative_humidity': 
    #    vars_mean_list.model_name.iat[index] = 'RH'
    #    vars_mean_list.conv_factor.iat[index] = 1/100.
    if standard_name=='eastward_wind': 
        vars_mean_list.model_name.iat[index] = 'CSP_U'
    if standard_name=='northward_wind': 
        vars_mean_list.model_name.iat[index] = 'CSP_V'
    if standard_name=='upward_air_velocity': 
        vars_mean_list.model_name.iat[index] = 'CSP_W_MASS'
    if standard_name=='air_potential_temperature': 
        vars_mean_list.model_name.iat[index] = 'CSP_TH'
    if standard_name=='air_liquid_potential_temperature': 
        vars_mean_list.model_name.iat[index] = 'CSP_THL'
    if standard_name=='specific_turbulent_kinetic_energy_resolved': 
        vars_mean_list.model_name.iat[index] = 'CSP_TKE_RS'
    if standard_name=='specific_turbulent_kinetic_energy_sgs': 
        vars_mean_list.model_name.iat[index] = 'CSP_TKE_SGS'
    if standard_name=='variance_of_upward_air_velocity': 
        vars_mean_list.model_name.iat[index] = 'CSP_W2_MASS'
    if standard_name=='mass_fraction_of_cloud_liquid_water_in_air': 
        vars_mean_list.model_name.iat[index] = 'CSP_QC'
    if standard_name=='mass_fraction_of_rain_water_in_air': 
        vars_mean_list.model_name.iat[index] = 'CSP_QR'
    if standard_name=='mass_fraction_of_cloud_ice_in_air': 
        vars_mean_list.model_name.iat[index] = 'CSP_QI'
    if standard_name=='mass_fraction_of_snow_in_air': 
        vars_mean_list.model_name.iat[index] = 'CSP_QS'
    if standard_name=='mass_fraction_of_graupel_in_air': 
        vars_mean_list.model_name.iat[index] = 'CSP_QG'
    if standard_name=='cloud_area_fraction_in_atmospheric_layer': 
        vars_mean_list.model_name.iat[index] = 'CSP_A_CC'
    if standard_name=='number_of_cloud_droplets_in_air': 
        vars_mean_list.model_name.iat[index] = 'CSP_QNC'
    if standard_name=='number_of_rain_droplets_in_air': 
        vars_mean_list.model_name.iat[index] = 'CSP_QNR'
    if standard_name=='number_of_cloud_ice_crystals_in_air': 
        vars_mean_list.model_name.iat[index] = 'CSP_QNI'
    if standard_name=='number_of_snow_crystals_in_air': 
        vars_mean_list.model_name.iat[index] = 'CSP_QNS'
    if standard_name=='number_of_graupel_crystals_in_air': 
        vars_mean_list.model_name.iat[index] = 'CSP_QNG'
    #if standard_name=='longwave_heating_rate_in_air': 
    #    vars_mean_list.model_name.iat[index] = 'HRlw'
    #if standard_name=='shortwave_heating_rate_in_air': 
    #    vars_mean_list.model_name.iat[index] = 'HRsw'
    print(standard_name,' <--- ',vars_mean_list.model_name.iat[index],' * ',vars_mean_list.conv_factor.iat[index])

air_pressure  <---  CSP_P  *  1.0
air_volumic_mass  <---  CSP_RHO  *  1.0
air_temperature  <---  missing data  *  1.0
specific_humidity  <---  CSP_QV  *  1.0
total_water_content  <---  CSP_QT  *  1.0
relative_humidity  <---  missing data  *  1.0
eastward_wind  <---  CSP_U  *  1.0
northward_wind  <---  CSP_V  *  1.0
upward_air_velocity  <---  CSP_W_MASS  *  1.0
air_potential_temperature  <---  CSP_TH  *  1.0
specific_turbulent_kinetic_energy_resolved  <---  CSP_TKE_RS  *  1.0
specific_turbulent_kinetic_energy_sgs  <---  CSP_TKE_SGS  *  1.0
variance_of_upward_air_velocity  <---  CSP_W2_MASS  *  1.0
mass_fraction_of_cloud_liquid_water_in_air  <---  CSP_QC  *  1.0
mass_fraction_of_rain_water_in_air  <---  CSP_QR  *  1.0
mass_fraction_of_cloud_ice_in_air  <---  CSP_QI  *  1.0
mass_fraction_of_snow_in_air  <---  CSP_QS  *  1.0
mass_fraction_of_graupel_in_air  <---  CSP_QG  *  1.0
cloud_area_fraction_in_atmospheric_layer  <---  CSP_A_CC  *  1.0
precipitation_flux_in_air  <---  missing data  *

### Create DEPHY output file

In [19]:
# create DEPHY output file

if os.path.exists(my_savedir + dephy_filename):
    os.remove(my_savedir + dephy_filename)
    print('The file ' + dephy_filename + ' has been deleted successfully')    
dephy_file = Dataset(my_savedir + dephy_filename,mode='w',format='NETCDF3_CLASSIC')

# create global attributes

dephy_file.title='WRF LES results for COMBLE-MIP case: fixed Nd and Ni'
dephy_file.reference='https://github.com/ARM-Development/comble-mip'
dephy_file.authors='Tim Juliano (tjuliano@ucar.edu)'
#dephy_file.source=input_filename
dephy_file.version=dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
dephy_file.format_version='DEPHY SCM format version 1.6'
dephy_file.script='convert_WRF_LES_output_to_dephy_format.ipynb'
dephy_file.startDate='2020-03-12T22:00:00Z'
dephy_file.force_geo=1
dephy_file.surfaceType='ocean (after spin-up)'
dephy_file.surfaceForcing='ts (after spin-up)'
dephy_file.lat=str(wrf_lat) + ' deg N'
dephy_file.dx=str(wrf_dx) + ' m'
dephy_file.dy=str(wrf_dy) + ' m'
dephy_file.dz='see zf variable'
dephy_file.nx=str(wrf_nx)
dephy_file.ny=str(wrf_ny)
dephy_file.nz=str(wrf_nz)

# create dimensions

nz = wrf_nz
zf = dephy_file.createDimension('zf', nz)
zf = dephy_file.createVariable('zf', np.float64, ('zf',))
zf.units = 'm'
zf.long_name = 'altitude'
zf[:] = wrf_snds['CSP_Z'][:,:].data.mean(axis=0)

nt = wrf_scas.dims['Time']
time = dephy_file.createDimension('time', nt)
time = dephy_file.createVariable('time', np.float64, ('time',))
time.units = 'seconds since ' + dephy_file.startDate
time.long_name = 'time'
# seconds between WRF samples
delt = 5.*60
time[:] = delt*wrf_scas['Time'].data

# create and fill variables

for index in vars_mean_list.index[2:]:
    std_name = vars_mean_list.standard_name.iat[index]
    var_name = vars_mean_list.variable_id.iat[index]
    mod_name = vars_mean_list.model_name.iat[index]
    c_factor = vars_mean_list.conv_factor.iat[index]
    if vars_mean_list.dimensions.iat[index]=='time':
        new_sca = dephy_file.createVariable(var_name, np.float64, ('time'))
        new_sca.units = vars_mean_list.units.iat[index]
        new_sca.long_name = std_name
        if vars_mean_list.model_name.iat[index]!='missing data':
            new_sca[:] = wrf_scas[mod_name].data*c_factor
    if vars_mean_list.dimensions.iat[index]=='time, height':
        new_snd = dephy_file.createVariable(var_name, np.float64, ('time','zf'))
        new_snd.units = vars_mean_list.units.iat[index]
        new_snd.long_name = std_name
        if vars_mean_list.model_name.iat[index]!='missing data': 
            new_snd[:] = wrf_snds[mod_name].data*c_factor

print(dephy_file)
dephy_file.close()

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: WRF LES results for COMBLE-MIP case: fixed Nd and Ni
    reference: https://github.com/ARM-Development/comble-mip
    authors: Tim Juliano (tjuliano@ucar.edu)
    version: 2023-06-27 16:45:04
    format_version: DEPHY SCM format version 1.6
    script: convert_WRF_LES_output_to_dephy_format.ipynb
    startDate: 2020-03-12T22:00:00Z
    force_geo: 1
    surfaceType: ocean (after spin-up)
    surfaceForcing: ts (after spin-up)
    lat: 74.0 deg N
    dx: 100.0 m
    dy: 100.0 m
    dz: see zf variable
    nx: 256
    ny: 256
    nz: 159
    dimensions(sizes): zf(159), time(241)
    variables(dimensions): float64 zf(zf), float64 time(time), float64 pa(time,zf), float64 rho(time,zf), float64 ta(time,zf), float64 qv(time,zf), float64 qt(time,zf), float64 hur(time,zf), float64 ua(time,zf), float64 va(time,zf), float64 wa(time,zf), float64 theta(time,zf), float64 tke_res(time,zf), float

### Check output file

In [20]:
dephy_check = xr.open_dataset(my_savedir + dephy_filename, decode_times=False)
dephy_check

<xarray.Dataset>
Dimensions:  (time: 241, zf: 159)
Coordinates:
  * zf       (zf) float64 10.61 34.47 63.6 ... 6.859e+03 6.929e+03 6.984e+03
  * time     (time) float64 0.0 300.0 600.0 900.0 ... 7.14e+04 7.17e+04 7.2e+04
Data variables:
    pa       (time, zf) float64 ...
    rho      (time, zf) float64 ...
    ta       (time, zf) float64 ...
    qv       (time, zf) float64 ...
    qt       (time, zf) float64 ...
    hur      (time, zf) float64 ...
    ua       (time, zf) float64 ...
    va       (time, zf) float64 ...
    wa       (time, zf) float64 ...
    theta    (time, zf) float64 ...
    tke_res  (time, zf) float64 ...
    tke_sgs  (time, zf) float64 ...
    hfss     (time) float64 ...
    hfls     (time) float64 ...
    ol       (time) float64 ...
    ustar    (time) float64 ...
    zi       (time) float64 ...
    ps       (time) float64 ...
    ts       (time) float64 ...
    w2       (time, zf) float64 ...
    ql       (time, zf) float64 ...
    qr       (time, zf) float64 ...
    qi       (time, zf) float64 ...
    qs       (time, zf) float64 ...
    qg       (time, zf) float64 ...
    cl       (time, zf) float64 ...
    clt      (time) float64 ...
    pr       (time) float64 ...
    prf      (time, zf) float64 ...
    pri      (time) float64 ...
    prfi     (time, zf) float64 ...
    cwp      (time) float64 ...
    rwp      (time) float64 ...
    iwp      (time) float64 ...
    swp      (time) float64 ...
    gwp      (time) float64 ...
    nt       (time, zf) float64 ...
    na       (time, zf) float64 ...
    nc       (time, zf) float64 ...
    nr       (time, zf) float64 ...
    ni       (time, zf) float64 ...
    ns       (time, zf) float64 ...
    ng       (time, zf) float64 ...
    cer      (time, zf) float64 ...
    cod      (time) float64 ...
    od       (time) float64 ...
    rlh      (time, zf) float64 ...
    rsh      (time, zf) float64 ...
    rld      (time, zf) float64 ...
    rlu      (time, zf) float64 ...
    rsd      (time, zf) float64 ...
    rsu      (time, zf) float64 ...
    rsdt     (time) float64 ...
    rsut     (time) float64 ...
    rlut     (time) float64 ...
    rlds     (time) float64 ...
    rlus     (time) float64 ...
    rsds     (time) float64 ...
    rsus     (time) float64 ...
Attributes:
    title:           WRF LES results for COMBLE-MIP case: fixed Nd and Ni
    reference:       https://github.com/ARM-Development/comble-mip
    authors:         Tim Juliano (tjuliano@ucar.edu)
    version:         2023-06-27 16:45:04
    format_version:  DEPHY SCM format version 1.6
    script:          convert_WRF_LES_output_to_dephy_format.ipynb
    startDate:       2020-03-12T22:00:00Z
    force_geo:       1
    surfaceType:     ocean (after spin-up)
    surfaceForcing:  ts (after spin-up)
    lat:             74.0 deg N
    dx:              100.0 m
    dy:              100.0 m
    dz:              see zf variable
    nx:              256
    ny:              256
    nz:              159